In [2]:
from sqlalchemy import create_engine

In [3]:
user = "schedulin"
passw = "MySQLIsFun"
host = "35.231.228.133"
database = "schedulin"

In [4]:
db = create_engine(
    'mysql+pymysql://{0}:{1}@{2}/{3}' \
        .format(user, passw, host, database), \
    connect_args = {'connect_timeout': 10})
conn = db.connect()

In [5]:
def query_db(query):
    results = conn.execute(query).fetchall()
    for r in results:
        print(r)

In [36]:
query = """
    SHOW TABLES;
"""
query_db(query)

('reservation',)
('resource',)
('timetable',)
('user',)


In [60]:
query = """
    SELECT 
        COUNT(*) 
    FROM reservation;
"""
query_db(query)

(29151,)


In [13]:
query = """
    SELECT 
        DISTINCT 
            type 
    FROM resource;
"""
query_db(query)

('PADEL',)
('PISCINA',)
('GIMNASIO',)
('SALA COMUN',)


In [14]:
query = """
    SELECT 
        type,
        price    
    FROM resource
    WHERE price = (
        SELECT 
            MAX(price) AS max
        FROM resource
        GROUP BY type
        ORDER BY MAX(price) DESC
        LIMIT 1);
"""
query_db(query)

('GIMNASIO', Decimal('5.00'))
('SALA COMUN', Decimal('5.00'))


In [16]:
query = """
    SELECT 
        MIN(start_time), 
        MAX(end_time) 
    FROM timetable AS t 
    LEFT JOIN resource AS r 
    ON t.id = r.timetable_id 
    WHERE r.type = 'PADEL';
"""
query_db(query)

('09:00', '22:30')


In [19]:
query = """
    SELECT 
        DISTINCT
            TIMEDIFF(
            STR_TO_DATE(end_time,'%%H:%%i'), 
            STR_TO_DATE(start_time,'%%H:%%i'))
    FROM timetable AS t 
    LEFT JOIN resource AS r 
    ON t.id = r.timetable_id 
    WHERE r.type = 'PADEL';
"""
query_db(query)

(datetime.timedelta(seconds=5400),)


In [20]:
query = """
    SELECT
        AVG(hours_in_advance) AS hours
    FROM resource;
"""
query_db(query)

(Decimal('6.5714'),)


In [21]:
query = """
    SELECT
        AVG(price) AS price
    FROM resource;
"""
query_db(query)

(Decimal('1.857143'),)


In [22]:
query = """
    SELECT
    --    COUNT(DISTINCT(email)) AS users
        COUNT(ID) AS users
    FROM user;
"""
query_db(query)

(380,)


In [23]:
query = """
    SELECT
        COUNT(ID) AS users
    FROM user
    WHERE status = 'INACTIVE';
"""
query_db(query)

(45,)


In [31]:
query = """
    SELECT 
        email
    FROM user
    WHERE deletion_date IS NOT NULL;
"""
query_db(query)

In [32]:
query = """
    SELECT
        email
    FROM user
    WHERE deletion_date IS NOT NULL
        AND status = 'ACTIVE';
"""
query_db(query)

In [33]:
query = """
    SELECT
        DATE(deletion_date),
        COUNT(deletion_date) AS total
    FROM user
    WHERE deletion_date IS NOT NULL
    GROUP BY DATE(deletion_date)
    LIMIT 1;
"""
query_db(query)

In [34]:
query = """
    SELECT
        u.id, 
        u.deletion_date, 
        r.date
    FROM (
        SELECT id, DATE(deletion_date) AS deletion_date
        FROM user
        WHERE deletion_date IS NOT NULL) AS u
    INNER JOIN (
        SELECT *
        FROM reservation) AS r
    ON u.id = r.user_id
        AND u.deletion_date < r.date
        AND (r.date IS NOT NULL
            OR r.date != 'NULL');
"""
query_db(query)

OperationalError: (pymysql.err.OperationalError) (1525, "Incorrect DATE value: 'NULL'")
[SQL: 
    SELECT
        u.id, 
        u.deletion_date, 
        r.date
    FROM (
        SELECT id, DATE(deletion_date) AS deletion_date
        FROM user
        WHERE deletion_date IS NOT NULL) AS u
    INNER JOIN (
        SELECT *
        FROM reservation) AS r
    ON u.id = r.user_id
        AND u.deletion_date < r.date
        AND (r.date IS NOT NULL
            OR r.date != 'NULL');
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [35]:
query = """
    SELECT 
        user_id,
        total
    FROM(
    SELECT 
        user_id,
        COUNT(user_id) AS total
    FROM reservation
    GROUP BY user_id) as t
    WHERE total = (
        SELECT
            MAX(total) AS MAX
        FROM (
            SELECT 
                user_id,
                COUNT(user_id) AS total
            FROM reservation
            GROUP BY user_id
            ORDER BY total DESC
            LIMIT 1) AS m);
"""
query_db(query)

(366, 99)


In [39]:
query = """
    SELECT
        status, 
        (COUNT(*) / (
            SELECT COUNT(*)
            FROM reservation)) * 100
    FROM reservation 
    GROUP BY status;
"""
query_db(query)

('CONFIRMED', Decimal('94.8818'))
('CANCELLED', Decimal('3.0942'))
('EXECUTED', Decimal('2.0239'))


In [50]:
query = """
    SELECT
        type,
        TOTAL
    FROM (
        SELECT 
            RE.type,
            COUNT(*) AS TOTAL
        FROM (
            SELECT 
                resource_id
            FROM reservation) AS R
        LEFT JOIN (
            SELECT
                id, type
            FROM resource) AS RE
        ON R.resource_id = RE.id
        GROUP BY RE.type) AS T
    WHERE TOTAL = (
        SELECT 
            MAX(TOTAL)
        FROM (
            SELECT 
                RE.type,
                COUNT(*) AS TOTAL
            FROM (
                SELECT 
                    resource_id
                FROM reservation) AS R
            LEFT JOIN (
                SELECT
                    id, type
                FROM resource) AS RE
            ON R.resource_id = RE.id
            GROUP BY RE.type) AS M);
"""
query_db(query)

('PADEL', 18585)


In [54]:
query = """
    SELECT
        type,
        TOTAL
    FROM (
        SELECT 
            RE.type,
            COUNT(*) AS TOTAL
        FROM (
            SELECT 
                resource_id
            FROM reservation) AS R
        LEFT JOIN (
            SELECT
                id, type
            FROM resource) AS RE
        ON R.resource_id = RE.id
        GROUP BY RE.type) AS T
    WHERE TOTAL = (
            SELECT 
                COUNT(*) AS TOTAL
            FROM (
                SELECT 
                    resource_id
                FROM reservation) AS R
            LEFT JOIN (
                SELECT
                    id, type
                FROM resource) AS RE
            ON R.resource_id = RE.id
            GROUP BY RE.type
            LIMIT 1);
"""
query_db(query)

('PADEL', 18585)


In [97]:
query = """
    SELECT
        start_time, end_time,
        COUNT(start_time) AS TOTAL
    FROM (
        SELECT id, resource_id
        FROM reservation) AS RE
    LEFT JOIN (
        SELECT id, timetable_id, type
        FROM resource) AS R
    ON RE.resource_id = R.id
    LEFT JOIN (
        SELECT id, start_time, end_time
        FROM timetable) AS T
    ON T.id = R.timetable_id
    GROUP BY start_time, end_time
    ORDER BY TOTAL DESC;
"""
query_db(query)

('10:00', '16:00', 4331)
('16:00', '22:00', 4119)
('10:00', '22:00', 2116)
('10:30', '12:00', 2100)
('15:00', '16:30', 2098)
('13:30', '15:00', 2083)
('09:00', '10:30', 2064)
('18:00', '19:30', 2062)
('16:30', '18:00', 2060)
('12:00', '13:30', 2057)
('21:00', '22:30', 2046)
('19:30', '21:00', 2015)


In [68]:
query = """
    SELECT 
       AVG(TOTAL) AS AVG
    FROM (
        SELECT
            date,
            COUNT(*) AS TOTAL
        FROM reservation
        GROUP BY date) AS R;
"""
query_db(query)

(Decimal('7.9910'),)


In [84]:
query = """
    SELECT
        G,
        COUNT(*) AS TOTAL
    FROM (
        SELECT 
            CASE
                WHEN STR_TO_DATE(start_time,'%%H:%%i')
                    >= STR_TO_DATE('16:00','%%H:%%i') THEN 'EVENING'
                ELSE 'MORNING'
            END AS 'G'
        FROM (
            SELECT id, resource_id
            FROM reservation) AS RE
        LEFT JOIN (
            SELECT id, timetable_id, type
            FROM resource) AS R
        ON RE.resource_id = R.id
        LEFT JOIN (
            SELECT id, start_time
            FROM timetable) AS T
        ON T.id = R.timetable_id) AS EM
    GROUP BY G
    ORDER BY TOTAL DESC;
"""
query_db(query)

('MORNING', 16849)
('EVENING', 12302)


In [93]:
query = """
    SELECT
        HAS_PROBLEMS,
        COUNT(*) AS TOTAL
    FROM (
        SELECT 
            CASE
                WHEN max_pax < num_pax THEN 1
                ELSE 0
            END AS 'HAS_PROBLEMS'
        FROM (
            SELECT id, resource_id, num_pax
            FROM reservation) AS RE
        LEFT JOIN (
            SELECT id, timetable_id, max_pax
            FROM resource) AS R
        ON RE.resource_id = R.id) AS M
        GROUP BY HAS_PROBLEMS
        ORDER BY TOTAL DESC;
"""
query_db(query)

(0, 19631)
(1, 9520)


In [115]:
query = """
    SELECT
        user_id,
        date,
        start_time,
        GROUP_CONCAT(DISTINCT type),
        COUNT(*) AS TOTAL
    FROM (
        SELECT user_id, date, start_time, type
        FROM (
            SELECT id, resource_id, user_id, date
            FROM reservation
            WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
        LEFT JOIN (
            SELECT id, timetable_id, type
            FROM resource) AS R
        ON RE.resource_id = R.id
        LEFT JOIN (
            SELECT id, start_time, end_time
            FROM timetable) AS T
        ON T.id = R.timetable_id) AS M
    GROUP BY user_id, date, start_time
    HAVING TOTAL > 1;
"""
query_db(query)

(3, datetime.date(2021, 6, 8), '13:30', 'PADEL', 2)
(18, datetime.date(2017, 11, 16), '16:00', 'GIMNASIO,PISCINA', 2)
(22, datetime.date(2019, 9, 21), '10:00', 'GIMNASIO,PISCINA', 2)
(33, datetime.date(2021, 10, 15), '10:00', 'GIMNASIO,SALA COMUN', 2)
(44, datetime.date(2016, 9, 21), '10:00', 'SALA COMUN', 2)
(44, datetime.date(2021, 5, 10), '16:30', 'PADEL', 2)
(45, datetime.date(2014, 6, 26), '19:30', 'PADEL', 2)
(53, datetime.date(2021, 8, 20), '10:00', 'PISCINA,SALA COMUN', 2)
(93, datetime.date(2020, 12, 1), '10:00', 'GIMNASIO,SALA COMUN', 2)
(103, datetime.date(2017, 5, 22), '09:00', 'PADEL', 2)
(119, datetime.date(2015, 4, 10), '19:30', 'PADEL', 2)
(123, datetime.date(2014, 5, 21), '10:00', 'SALA COMUN', 2)
(130, datetime.date(2013, 2, 27), '13:30', 'PADEL', 2)
(132, datetime.date(2013, 7, 11), '16:00', 'GIMNASIO', 2)
(159, datetime.date(2018, 10, 22), '10:30', 'PADEL', 2)
(171, datetime.date(2022, 8, 28), '10:00', 'GIMNASIO,PISCINA', 2)
(217, datetime.date(2015, 6, 26), '10:00'

In [14]:
query = """
        SELECT user_id, date, start_time, end_time, type
        FROM (
            SELECT id, resource_id, user_id, date
            FROM reservation
            WHERE status IN ('CONFIRMED', 'EXECUTED')
                AND user_id = 377
                AND date = '2014-11-19') AS RE
        LEFT JOIN (
            SELECT id, timetable_id, type
            FROM resource) AS R
        ON RE.resource_id = R.id
        LEFT JOIN (
            SELECT id, start_time, end_time
            FROM timetable) AS T
        ON T.id = R.timetable_id
        ORDER BY start_time ASC
"""
query_db(query)

(377, datetime.date(2014, 11, 19), '10:00', '16:00', 'PISCINA')
(377, datetime.date(2014, 11, 19), '13:30', '15:00', 'PADEL')


In [22]:
# https://www.mysqltutorial.org/mysql-window-functions/mysql-lag-function/

query = """
    SELECT *
    FROM (
        SELECT  user_id, date, start_time, end_time, post_start_time,
            CASE 
                WHEN end_time > post_start_time THEN 1
                ELSE 0
            END HAS_DUP
        FROM (
            SELECT user_id, date, start_time, end_time,
                LAG(start_time, 1) OVER (
                    PARTITION BY user_id, date
                    ORDER BY start_time DESC) AS post_start_time
            FROM (
                SELECT id, resource_id, user_id, date
                FROM reservation
                WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
            LEFT JOIN (
                SELECT id, timetable_id, type
                FROM resource) AS R
            ON RE.resource_id = R.id
            LEFT JOIN (
                SELECT id, start_time, end_time
                FROM timetable) AS T
            ON T.id = R.timetable_id
            ORDER BY user_id DESC, date DESC, start_time DESC) AS M
        WHERE post_start_time IS NOT NULL) AS X
    WHERE HAS_DUP = 1;
"""
query_db(query)

(377, datetime.date(2014, 11, 19), '10:00', '16:00', '13:30', 1)
(373, datetime.date(2022, 2, 3), '10:00', '16:00', '15:00', 1)
(371, datetime.date(2021, 4, 13), '10:00', '16:00', '10:00', 1)
(365, datetime.date(2022, 7, 22), '10:00', '22:00', '15:00', 1)
(365, datetime.date(2019, 9, 19), '10:00', '22:00', '19:30', 1)
(359, datetime.date(2013, 2, 20), '13:30', '15:00', '13:30', 1)
(356, datetime.date(2017, 3, 6), '10:00', '22:00', '21:00', 1)
(355, datetime.date(2022, 3, 5), '10:00', '22:00', '21:00', 1)
(348, datetime.date(2022, 2, 17), '10:00', '16:00', '15:00', 1)
(348, datetime.date(2019, 12, 10), '10:00', '16:00', '15:00', 1)
(344, datetime.date(2018, 8, 3), '15:00', '16:30', '16:00', 1)
(342, datetime.date(2017, 9, 22), '10:00', '16:00', '10:00', 1)
(341, datetime.date(2018, 11, 16), '09:00', '10:30', '10:00', 1)
(329, datetime.date(2013, 3, 22), '10:30', '12:00', '10:30', 1)
(325, datetime.date(2020, 12, 28), '15:00', '16:30', '15:00', 1)
(323, datetime.date(2020, 2, 20), '16:30

In [29]:
query = """

            SELECT user_id, SUM(price) AS TOTAL
            FROM (
                SELECT id, resource_id, user_id, date
                FROM reservation
                WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
            LEFT JOIN (
                SELECT id, price
                FROM resource) AS R
            ON RE.resource_id = R.id
            GROUP BY user_id
            ORDER BY TOTAL DESC
            LIMIT 10
"""
query_db(query)

(298, Decimal('201.50'))
(197, Decimal('193.25'))
(123, Decimal('190.50'))
(119, Decimal('190.00'))
(117, Decimal('187.75'))
(69, Decimal('187.75'))
(318, Decimal('187.00'))
(2, Decimal('182.00'))
(304, Decimal('181.75'))
(292, Decimal('178.50'))


In [30]:
query = """

            SELECT user_id, SUM(price) AS TOTAL
            FROM (
                SELECT id, resource_id, user_id, date
                FROM reservation
                WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
            LEFT JOIN (
                SELECT id, price
                FROM resource) AS R
            ON RE.resource_id = R.id
            GROUP BY user_id
            ORDER BY TOTAL ASC
            LIMIT 10
"""
query_db(query)

(86, Decimal('77.00'))
(370, Decimal('85.25'))
(34, Decimal('87.75'))
(160, Decimal('90.50'))
(85, Decimal('96.25'))
(51, Decimal('96.25'))
(338, Decimal('98.50'))
(287, Decimal('98.50'))
(244, Decimal('100.75'))
(301, Decimal('101.50'))


In [46]:
query = """
    SELECT id, timetable_id, price, max_pax
    FROM resource
    WHERE type = 'GIMNASIO'
"""
query_db(query)

(12, 11, Decimal('3.00'), 6)
(13, 12, Decimal('5.00'), 6)


In [42]:
query = """
    SELECT id, start_time, end_time
    FROM timetable
    WHERE id = 10
"""
query_db(query)

(10, '10:00', '22:00')


In [50]:
query = """
SELECT Y.TOTAL + Z.TOTAL
FROM (
    SELECT SUM(price) * 365 AS TOTAL
    FROM (
        SELECT id, price
        FROM resource
        WHERE type IN ('PADEL', 'SALA COMUN')) AS R) AS Y
LEFT JOIN (
    SELECT SUM(price * max_pax) * 365 AS TOTAL
    FROM (
        SELECT id, price, max_pax
        FROM resource
        WHERE type = 'GIMNASIO') AS P) AS Z
ON 1 = 1;
"""
query_db(query)

(Decimal('24090.00'),)


In [67]:
query = """
SELECT SUM(TOTAL)
FROM (
    SELECT SUM(price) * 365 AS TOTAL
    FROM (
        SELECT id, price
        FROM resource
        WHERE type IN ('PADEL', 'SALA COMUN')) AS R
    UNION
    SELECT SUM(price * max_pax) * 365 AS TOTAL
    FROM (
        SELECT id, price, max_pax
        FROM resource
        WHERE type = 'GIMNASIO') AS P) AS Z;
"""
query_db(query)

(Decimal('24090.00'),)


In [65]:
query = """

SELECT
    AVG(TOTAL)
FROM (
    SELECT
        YEAR(date) AS YEAR, 
        SUM(price) AS TOTAL
    FROM (

        SELECT date, price
        FROM (
            SELECT id, resource_id, date
            FROM reservation
            WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
        LEFT JOIN (
            SELECT id, price
            FROM resource) AS R
        ON RE.resource_id = R.id) AS R
    GROUP BY YEAR(date)) AS D;
"""
query_db(query)

(Decimal('4772.068182'),)


In [75]:
query = """
        SELECT 
            date, type, SUM(price), COUNT(date)
        FROM (
            SELECT id, resource_id, date
            FROM reservation
            WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
        LEFT JOIN (
            SELECT id, price, type
            FROM resource) AS R
        ON RE.resource_id = R.id
        GROUP BY date, type
        ORDER BY date DESC, type DESC
        LIMIT 10;
"""
query_db(query)

(datetime.date(2022, 11, 4), 'SALA COMUN', Decimal('5.00'), 1)
(datetime.date(2022, 11, 4), 'PISCINA', Decimal('0.00'), 3)
(datetime.date(2022, 11, 4), 'PADEL', Decimal('7.00'), 6)
(datetime.date(2022, 11, 4), 'GIMNASIO', Decimal('5.00'), 1)
(datetime.date(2022, 11, 3), 'PADEL', Decimal('3.00'), 3)
(datetime.date(2022, 11, 3), 'GIMNASIO', Decimal('14.00'), 4)
(datetime.date(2022, 11, 2), 'PADEL', Decimal('13.25'), 11)
(datetime.date(2022, 11, 2), 'GIMNASIO', Decimal('6.00'), 2)
(datetime.date(2022, 11, 1), 'SALA COMUN', Decimal('5.00'), 1)
(datetime.date(2022, 11, 1), 'PISCINA', Decimal('0.00'), 2)
